<a href="./">Go to this directory index</a>

Contents

  - [Ammonia2: iterative solver for aqua-ammonia absorption cycle¶](#Ammonia2:-iterative-solver-for-aqua-ammonia-absorption-cycle)

In [2]:
%%html
<img src="../img/Diagram_for_ammonia.svg"><!-- In source distro-->
<img src="./img/Diagram_for_ammonia.svg"><!-- In web docs -->

# Ammonia2: iterative solver for aqua-ammonia absorption cycle

The previous ammonia-water cycle model (`import ammonia1`) solved the cycle by inputing internal temperatures at the endpoints of heat exchange processes, then solved for UA by the heat exchanger integral equation. Suppose instead we want to input heat exchanger and pump specifications for the system above, then solve for the internal temperatures. How then do we solve the cycle?

First let's compare inputs to recognize where we might need to change assumptions.

<table>
<tr>
<th>Set of inputs for easy update (ammonia1)</th>
<th>Set of inputs for build specification</th>
<th>Set of inputs for operating spec.</th>
<th>Equivalent input for ammonia2</th>
</tr>
<tr>
<td>
$T_{cond}$<br>
$T_{evap}$<br>
$T_{abs}$<br>
$T_{gen}$<br>
$T_{rect}$<br>
$\dot{m}$<br>
Subcooling $\Delta T_{cond}$<br>
Superheat $\Delta T_{evap}$<br>
Assume $P_{evap}$ for absorber properties
</td>
<td>
$UA_{cond}$<br>
$UA_{evap}$<br>
$UA_{abs}$<br>
$UA_{gen}$<br>
$UA_{rect}$<br>
Pump curve<br>
Solution valve minor loss coeffcient<br>
Refrigerant valve minor loss coefficient<br>
Absorption mass transfer resistance
</td>
<td>
"<br>"<br>"<br>"<br>"<br>
Pump power input<br>
(Adjustable)<br>
(Adjustable)<br>
"
</td>
<td>
"<br>"<br>"<br>"<br>"<br>
$\dot{m}$<br>
Assume $P_{cond}$ calculated from $T_{cond}$<br>
Assume $P_{evap}$ calculated from $T_{evap}$<br>
Assume $P_{evap}$ for absorber properties
</td>
</tr>
<tr><th>Ammonia1 outputs</th><th></th><th></th><th>Ammonia2 outputs</th></tr>
<tr>
<td>Pressure drop from evaporator to absorber</td>
<td></td>
<td></td>
<td>Equilibrium pressure in absorber</td>
</tr>
</table>

Presuming that we want the models to behave similarly, the valve settings and pressures will need to be determined by the equilibrium pressure based on temperatures in the condenser (for high pressure) and evaporator (for low pressure). This is a potential pitfall; what do we do when the temperature in the ... . 

But let's proceed and see what happens. First, we give the program a basic structure:

```python
def program(initial_condition,time_to_run):
    initialize(initial_condition)
    while time < time_to_run:
        update()
```

The initial condition is needed to evaluate properties for the first time step, and looks like an input to `ammonia1`, ie. specifies state points throughout the system. During the update, since there are multiple types of variables evolving, we need to propose how to update them. A very simple procedure would be:

```python
def update():
    for component in heat_exchangers:
        component.fdm_update() # Using finite difference method for heat exchange,
                               # sets enthalpies, mass fractions, and flow rates
                               # for each outlet port.
    update_pressures()
```



In [9]:
from ammonia1 import AmmoniaChiller
ch = AmmoniaChiller()
ch

,T,P,x,h,s,u,v,Qu
rich_abs_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
rich_pump_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
rich_shx_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
rich_gen_sat_liquid,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
weak_gen_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
weak_shx_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
weak_exp_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
gen_vapor_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
gen_reflux_inlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665
refrig_rect_outlet,400,10,0.5,1107.51,3.63631,1014.08,0.0934303,0.507665


In [ ]:
class amm2_absorber_stream:
    def __init__():
        pass
    def q():
        pass
    def c
        

def amm2_get_absorber_stream(ch):
    

def amm2_update(ch,UA_abs,UA_gen,UA_cond,UA_evap,UA_rect):
    # Absorber
    # SHX
    # Generator
    # Rectifier
    # Condenser
    # Evaporator
    # CEHX
    
    
    

In [14]:
from numpy import inf


ZeroDivisionError: division by zero

# Table of contents generator

In [6]:
%%html
<pre id="TOC-markdown">TOC will be here</pre>
<script>
$("#TOC-markdown").html(
    $('h1,h2,h3,h4').filter(":has(a)").map(function(){return "  ".repeat($(this).prop("tagName")[1]) + "- ["+$(this).text()+"](" + $(this).children().attr("href") + ")";}).get().join("\n")
    );
</script>